# FEM code for linear elasticity

In [326]:
infile = open("mesh_1.inp", "r")
outfile = open("result_1.out", "w")

IOStream(<file result_1.out>)

## Input FEM data

This function reads the input file for the FEM analysis of two dimensional elasticty using three, and eight node isoparameteric elements:

Variable and array list:
--------------------------------
    npoin: Total number of nodes in the solution
    nelem: Total number of elements
    nvfix: Total number of constrained nodes
    ntype: Solution type(ntype  = 1, plane-stress and ntypes = 2 plain-strain).
    nnode: Number of nodes per element.
    ndofn: Number of degree of freedom (DOF) per node
    ndime: Number of spatial dimensions.
    ngaus: The order of numerical intergration.
    nmats: Total number of different materials in the solution.
    nstre: Number of stress components.
    nprop: Number of material properties.
    matno(nelem): Material types for the elements.
    nofix(nvfix): Node numbers at which one or more DOFs are constrained.
    lnods(nelem,nnode): Element nodal connectivity list.
    coord(npoin,ndime): Cartesian coordinates of each node.
    iffix(nvfix,ndofn): List of constrained DOFs.
    fixed(nvfix,ndofn): Prescribed value of any constrained DOFs.
    props(nmats,nprops): For each different material, the properties of that material.
    
    

In [327]:
function input_fem_elast(infile)
    header = split(chomp(readline(infile)))
    npoin, nelem, nvfix, ntype, nnode, ndofn, ndime, ngaus, nstre, nmats, nprop = parse.(Int, header)
    lnods = zeros(Int64,nelem,nnode)
    matno = zeros(Int64,nelem)
    coord = zeros(npoin,ndime)
    nofix = zeros(Int64,nvfix)
    iffix = zeros(Int64,nvfix, ndofn)
    fixed = zeros(nvfix, ndofn)
    props = zeros(nmats, nprop)
    jelem = 0
    
    for ielem = 1:nelem
        eleminf = split(chomp(readline(infile)))
        data = parse.(Int, eleminf)
        jelem = data[1]
        lnods[jelem,:] = data[2:nnode+1]
        matno[jelem] = data[nnode+2]
    end
        
    # Nodal coordinates
    for ipoin = 1:npoin
        point = split(chomp(readline(infile)))
        jpoin = parse(Int, point[1])
        coord[ipoin,:] = parse.(Float64,point[2:3])
    end
    # Constraint nodes and their values:
    for ivfix = 1:nvfix
        fix = split(chomp(readline(infile)))
        nofix[ivfix] = parse(Int,fix[1])
        dummy1 = parse.(Int, fix[2:3])
        dummy2 = parse.(Float64, fix[3:4])
        for idofn = 1 : ndofn
            iffix[ivfix,idofn] = dummy1[idofn]
            fixed[ivfix,idofn] = dummy2[idofn]
        end
    end
    
    # material properties of the system
    for imats = 1:nmats
        mat = split(chomp(readline(infile)))
        jmats = parse(Int, mat[1])
        props[jmats,1:2] .= parse.(Float64, mat[3:end])
    end
    # this is because the file is not read after this point
    
    println("***********************")
    println("*   FEM input data    *")
    println("***********************")
    println("Number of Elements:    ", nelem)
    println("Number of Nodes:       ", npoin)
    println("Number of fixed nodes: ", nvfix)
    println("Number of nodes per elem: ", nnode)
    println("Number of integration points: ", ngaus)
    println("Number of materials: ", nmats)
    println("Number of properties: ", nprop)
    
    return  (npoin, nelem, nvfix, ntype, nnode, ndofn, ndime, ngaus, nstre, nmats, nprop, lnods, matno,coord,props,
            nofix, iffix, fixed) 
end    

input_fem_elast (generic function with 1 method)

This function evaluates the position of samplnig points and associated weights for chosen order of numerical integration.

In [328]:
function gauss(ngaus, nnode)
    posgp = zeros(2*ngaus)
    weigp = zeros(ngaus)
    
    if (nnode == 3)
        if (ngaus == 1)
            posgp[1] = 1.0/3.0
            posgp[2] = 1.0/3.0
            weigp[1] = 0.5
        end
        
        if (ngaus == 3)
            posgp[1] = 0.5
            posgp[2] = 0.5
            posgp[3] = 0.0
            
            posgp[4] = 0.0
            posgp[5] = 0.5
            posgp[6] = 0.5
            
            weigp[1] = 1.0/6.0
            weigp[2] = 1.0/6.0
            weigp[3] = 1.0/6.0    
        end
        
        if (ngaus == 7)
            posgp[1] = 0.0
            posgp[2] = 0.5
            posgp[3] = 1.0
            posgp[4] = 0.5
            posgp[5] = 0.0
            posgp[6] = 0.0
            posgp[7] = 1.0/3.0
            
            posgp[8] = 0.0
            posgp[9] = 0.0
            posgp[10] = 0.0
            posgp[11] = 0.5
            posgp[12] = 1.0
            posgp[13] = 0.5
            posgp[14] = 1.0/3.0
            
            weigp[1] = 1.0/40.0
            weigp[2] = 1.0/15.0
            weigp[3] = 1.0/40.0
            weigp[4] = 1.0/15.0
            weigp[5] = 1.0/40.0
            weigp[6] = 1.0/15.0
            weigp[7] = 9.0/40.0
            
        end
    end
        
    if (nnode != 3)
        if(ngaus == 2)
            posgp[1] = -0.57735026918963
            weigp[1] = 1.0
        end
        if(ngaus > 2)
            posgp[1] = -0.7745966241483
            posgp[2] = 0.0
            weigp[1] = 0.55555555555556
            weigp[2] = 0.88888888888889
        end
        kgaus =  Int(floor(ngaus/2))
        print(kgaus)
        for igash = 1:kgaus
            jgash = ngaus+1-igash
            posgp[jgash] = -posgp[igash]
            weigp[jgash] = weigp[igash]
        end
    end   
    return posgp, weigp
end


gauss (generic function with 1 method)

In [329]:
npoin, nelem, nvfix, ntype, nnode, ndofn, ndime, ngaus, nstre, nmats, nprop, lnods, matno, coord, props, nofix, iffix, fixed = input_fem_elast(infile)
ntotc = npoin * ndofn
posgp, weigp = gauss(ngaus, nnode)
# Form global stifness matrix (lhs)
using NBInclude
@nbinclude("stiffness.ipynb")
gstif = stiffness(npoin, nelem, nnode, nstre, ndime, ndofn, 
    ngaus, ntype, lnods, matno, coord, props, posgp, weigp)

***********************
*   FEM input data    *
***********************
Number of Elements:    9
Number of Nodes:       40
Number of fixed nodes: 14
Number of nodes per elem: 8
Number of integration points: 3
Number of materials: 1
Number of properties: 2
1

80×80 Array{Float64,2}:
  0.733135  -0.170872  -1.28876   …   0.0        0.0        0.0
 -0.170872   1.7796     0.445821      0.0        0.0        0.0
 -1.28876    0.445821   2.86387       0.0        0.0        0.0
  0.477872  -3.10292   -1.01445       0.0        0.0        0.0
  0.50295   -0.181588  -1.12985       0.0        0.0        0.0
 -0.187998   1.21147    0.390156  …   0.0        0.0        0.0
  0.0        0.0        0.0           0.0        0.0        0.0
  0.0        0.0        0.0           0.0        0.0        0.0
  0.0        0.0        0.0           0.0        0.0        0.0
  0.0        0.0        0.0           0.0        0.0        0.0
  0.0        0.0        0.0       …   0.0        0.0        0.0
  0.0        0.0        0.0           0.0        0.0        0.0
  0.0        0.0        0.0           0.0        0.0        0.0
  ⋮                               ⋱                        
  0.0        0.0        0.0           0.0        0.0        0.0
  0.0        0.0    

This function integrates the prescribed distributed external loads at the element edges
and also the prescribed point loads at the nodes into the global force vector. 
Variable and array list:
--------------------------------

    npoin: Total number of nodes in the solution
    nelem: Total number of element in the solution
    ndofn: Number of DOFs per node.
    ngaus: The order of numerical integration.
    ndime: Number of global Cartesian coordinates.
    eload(nelem,nevab): Element force vector(nevab = nnode * ndofn)
    

In [330]:
function loads(infile,npoin,nelem,ndofn,nnode,ngaus,ndime,posgp,weigp,lnods,coord)
    # Initialize global force vector & elements loads
    nevab = nnode*ndofn
    ntotv = npoin*ndofn
    
    for itotv = 1:ntotv
        gforce = zeros(itotv,1)
    end
    eload = zeros(nelem,nevab)
    for ielem = 1:nelem
        for ievab = 1:nevab
            eload[ielem,ievab] = 0.0
        end
    end
    
    

    # loading types
    
    iplod, nedge = parse.(Int, split(chomp(readline(infile))))
    println(iplod, nedge )    
    # point forces:
    if (iplod != 0)
        pointdat = split(chomp(readline(infile)))
        println(pointdat)
        nplod = parse(Int, pointdat[1])
        lodpt = parse(Int,pointdat[2])
        dummy = parse.(Int,pointdat[3:4])
        point = zeros(ndofn)
        for idofn = 1:ndofn
            point[idofn] = dummy[idofn]
        end
        
        for ielem = 1:nelem
            for inode = 1:nnode
                nloca = lnods[ielem,inode]
                if (lodpt == nloca)
                    for idofn = 1:ndofn
                        nposi = (inode - 1)*ndofn + idofn
                        eload[ielem,nposi] = point[idofn]
                    end
                end
            end
        end
    end
    
    # Distributed forces
    if (nedge != 0)
        println("Number of loaded edges: ",nedge)
        println("List of loaded edges and applied loads:")
        for iedge = 1 : nedge
            nodeg = 3
            if (nnode != 8)
                nodeg = 2
            end
            press = zeros(nodeg,ndofn)
            # Reload loads
            loadsdat = split(chomp(readline(infile)))
            println("load: ", loadsdat)
            neass = parse(Int,loadsdat[1])
            noprs = parse.(Int,loadsdat[2:1+nodeg])
            
            for iodeg = 1 : nodeg
                dummy = split(chomp(readline(infile)))
                println("dummy :", dummy)
                press[iodeg,:] = parse.(Float64, dummy)
            end
            
            # Print
            println()
            println(neass)
            for iodeg = 1:nodeg
                println(noprs[iodeg])
            end
            
            println()
            for iodeg = 1:nodeg
                for idofn = 1:ndofn
                    println(press[iodeg,idofn])
                end
            end
            println()
            
            # end of reading
            #--Integrate along the edges
            etasp = -1.0
            
            elcod = zeros(ndime,nodeg)
            
            for iodeg = 1:nodeg
                lnode = noprs[iodeg]
                for idime = 1:ndime
                    elcod[idime,iodeg] = coord[lnode,idime]
                end
            end
            
            pgash = zeros(ndofn)
            dgash = zeros(ndofn)
            
            for igaus = 1:ngaus
                exisp = posgp[igaus]
                shape, deriv = sfr2(exisp,etasp,nnode)
                
                for idofn = 1 : ndofn
                    pgash[idofn] = 0.0
                    dgash[idofn] = 0.0
                    for iodeg = 1:nodeg
                        pgash[idofn] = pgash[idofn]+press[iodeg,idofn]*shape[iodeg]
                        dgash[idofn] = dgash[idofn]+elcod[idofn,iodeg]*deriv[1,iodeg]
                    end
                end
                
                dvolu = weigp[igaus]
                pxcom = dgash[1]*pgash[2]-dgash[2]*pgash[1]
                pycom = dgash[1]*pgash[1]+dgash[2]*pgash[2]
                
                for inode = 1:nnode
                    nloca = lnods[neass,inode]
                    if(nloca == noprs[1])
                        jnode = inode + nodeg - 1
                        kount = 0
                        for knode = inode : jnode
                            kount = kount + 1
                            ngash = (knode - 1) *ndofn + 1
                            mgash = (knode - 1) *ndofn + 2
                            if (knode > nnode)
                                ngash = 1
                                mgash = 2
                            end
                            eload[neass,ngash] = eload[neass,ngash]+pxcom*dvolu*shape[kount]
                            eload[neass,mgash] = eload[neass,mgash]+pycom*dvolu*shape[kount]
                        end
                    end
                end
            end
        end
    end
    
    
    # print nodal forces
    println()
    println("Nodal forces for elements:")
    for ielem = 1:nelem
        println("Element No: ",ielem)
        for ievab = 1:nevab
            println(eload[ielem,ievab])
            if ((nnode==8) && (ievab == nevab/2))
                println()
            end
        end
        println()
    end
    
    gforce = zeros(ntotv)
    # Generate global force vector
    for ielem =  1 : nelem
        for inode = 1 : nnode
            lnode = lnods[ielem,inode]
            for idofn = 1:ndofn
                itotv = (lnode-1)*ndofn+idofn
                ievab = (inode-1)*ndofn+idofn
                gforce[itotv] = gforce[itotv]+eload[ielem,ievab]
            end
        end
    end
    return gforce
end

loads (generic function with 1 method)

This function rearranges the stiffness matrix and force vector for prescribed boundary conditions.

In [331]:
function boundary_cond(npoin,nvfix,nofix,iffix,fixed,ndofn,gstif,gforce)
    ntotv = npoin*ndofn
    for ivfix = 1 : nvfix
        lnode = nofix[ivfix]
        
        for idofn = 1 : ndofn
            if(iffix[ivfix,idofn] == 1)
                itotv = (lnode - 1)*ndofn+idofn
                for jtotv = 1:ntotv
                    gstif[itotv,jtotv] = 0.0
                end
                
                gstif[itotv,itotv] = 1.0
                println("fixed :", fixed)
                gforce[itotv] = fixed[ivfix,idofn]
            end
        end
    end
    return gstif, gforce
end

boundary_cond (generic function with 1 method)

## Force vector and Boundary Conditions

In [332]:
gforce = loads(infile, npoin, nelem, ndofn, nnode, ngaus, ndime, posgp, weigp, lnods, coord)
gstif, gforce = boundary_cond(npoin, nvfix, nofix, iffix, fixed, ndofn, gstif, gforce)

03
Number of loaded edges: 3
List of loaded edges and applied loads:
load: SubString{String}["1", "12", "8", "1"]
dummy :SubString{String}["10.0", "0.0"]
dummy :SubString{String}["10.0", "0.0"]
dummy :SubString{String}["10.0", "0.0"]

1
12
8
1

10.0
0.0
10.0
0.0
10.0
0.0

load: SubString{String}["4", "23", "19", "12"]
dummy :SubString{String}["10.0", "0.0"]
dummy :SubString{String}["10.0", "0.0"]
dummy :SubString{String}["10.0", "0.0"]

4
23
19
12

10.0
0.0
10.0
0.0
10.0
0.0

load: SubString{String}["7", "34", "30", "23"]
dummy :SubString{String}["10.0", "0.0"]
dummy :SubString{String}["10.0", "0.0"]
dummy :SubString{String}["10.0", "0.0"]

7
34
30
23

10.0
0.0
10.0
0.0
10.0
0.0


Nodal forces for elements:
Element No: 1
3.3449996105413575
0.012499998544624025
0.0
0.0
0.0
0.0
0.0
0.0

0.0
0.0
0.0
0.0
-1.7825003382294267
0.4062498064349733
12.500000727688139
3.350000195020421

Element No: 2
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0

0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0

Element No: 3
0.0
0.0
0.0
0.0
0

([0.7331346665768209 -0.17087201737950344 … 0.0 0.0; 0.0 1.0 … 0.0 0.0; … ; 0.0 0.0 … 1.0 0.0; 0.0 0.0 … -1.839352105298027 4.506685210098872], [3.3449996105413575, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])

## Solve displacements

In [336]:
asdis = gstif\gforce

80-element Array{Float64,1}:
 81.1888553018406
 -7.869342335684846e-15
 44.945349636645226
 -2.7765755707422814e-14
 34.487035349867526
  1.2123963310229174e-14
 26.811227405144074
 -1.2045484353937108e-13
 29.61785546100352
  1.3539774060160833e-15
 24.825576774707244
 -1.3632079060911463e-14
 23.847667333386553
  ⋮
  0.0
 15.844764660614018
  0.0
 22.683166184661747
  0.0
 22.29736488055278
  0.0
 26.925004375834167
  0.0
 28.00430362338565
  0.0
 33.09497094800724

This function calculates the strain and stress values at the integration points of the elements resulting from the displacements.

In [354]:
function stress(asdis, nelem, npoin, nnode, ngaus,nstre,props,ntype,ndofn,ndime,lnods,matno,coord,posgp,weigp)
    ngaus2 = ngaus
    
    if(nnode == 3)
        ngaus2 = 1
    end
    
    mgaus = ngaus * ngaus2
    nevab = nnode*ndofn
    
    elem_stres = zeros(nelem, mgaus, nstre+1)
    
    for ielem = 1 : nelem
        
        # Material parameters and elasticity matrix
        mtype = matno[ielem]
        dmatx = modps(mtype,ntype,nstre,props)
        poiss = props[mtype,2]
        
        eldis = zeros(ndofn, nnode)
        elcod = zeros(ndofn, nnode)
        # Nodal displacements
        for inode = 1:nnode
            lnode = lnods[ielem,inode]
            for idofn = 1:ndofn
                nposn = (lnode - 1)*ndofn + idofn
                eldis[idofn,inode] = asdis[nposn]
                elcod[idofn,inode] = coord[lnode,idofn]
            end
        end
        
        # Integrate stresses
        kgasp = 0
        for igaus = 1:ngaus
            for jgaus = 1:ngaus
                
                kgasp = kgasp + 1
                exisp = posgp[igaus]
                etasp = posgp[jgaus]
                
                if (ngaus == 1)
                    etasp = posgp[ngaus+igaus]
                end
                
                shape, deriv = sfr2(exisp, etasp, nnode)
                
                cartd, djacb, gpcod = jacob2(ielem, elcod, kgasp, shape, deriv, nnode, ndime)
                
                bmatx = bmats(nevab,nstre,cartd, shape, nnode)
                
                stran = zeros(nstre)
                # Calculate the strains
                for istre = 1:nstre
                    stran[istre] = 0.0
                    for inode = 1 : nnode
                        for idofn = 1 : ndofn
                            ievab = (inode-1)*ndofn+idofn
                            stran[istre] = stran[istre] + bmatx[istre,ievab] * eldis[idofn,inode]
                        end
                    end
                end
                
                stres = zeros(nstre+1)
                # Calculate stresses
                for istre = 1:nstre
                    stres[istre] = 0.0
                    for jstre = 1:nstre
                        stres[istre] = stres[istre]+dmatx[istre,jstre]*stran[jstre]
                    end
                end
                
                if (ntype == 1)
                    stres[4] = 0.0
                end
                if (ntype == 2)
                    stres[4] = poiss*(stres[1] + stres[2])
                end
                
                for istre = 1 : nstre + 1
                    elem_stres[ielem,kgasp,istre] = stres[istre]
                end
                
            end
        end
    end
    return elem_stres  
end    

stress (generic function with 1 method)

## Calculate stresses

In [355]:
elem_stress = stress(asdis, nelem, npoin, nnode, ngaus, nstre, props, ntype, 
 ndofn, ndime, lnods, matno, coord, posgp, weigp)

9×9×4 Array{Float64,3}:
[:, :, 1] =
 -7.02806    -3.1746     -1.2161      …  0.87023    1.23711   1.47677
 -1.15651    -1.14962    -0.547724       1.30036    0.777074  0.917729
 -0.194553    0.0121937  -0.0293978      0.398096   0.29087   0.696179
 -2.63628    -1.42151    -0.00209556     1.62789    1.92426   0.499454
 -0.778531   -0.276107    0.102815       1.44408    0.177723  0.689796
 -0.0215677   0.25468     0.325253    …  1.03741   -0.175715  0.439793
 -1.17466    -0.750346    1.21281        2.058      0.101423  2.28737
  0.266751    0.371295    0.727852       1.28155   -0.33719   0.941196
  0.42316     0.573511    0.721797       0.881211  -0.480404  0.773932

[:, :, 2] =
 0.884883   1.12499    2.90265    0.955908  …  2.07643   -1.36566    0.351269
 1.21892    0.861882   1.70758    1.08109      1.43572   -0.861379   0.272863
 0.796588   0.845586   1.16615    0.629131     0.716563  -0.492978   0.145064
 2.65393   -0.149578   0.969394   1.31242      2.02841   -1.52769    1.27947
 0.

## Output results

This function prints out the solution results to file in tabulated form and in vtk file format for viewing results by using Paraview.     

In [374]:
using NBInclude
@nbinclude("output_fem.ipynb")
output_fem(npoin, nelem, nnode, lnods, coord, ndofn, ngaus, nstre, asdis, elem_stress)
print("Done")


*********************
*      OUTPUTS      *
*********************

Displacements:
Node number X-disp Y-disp
181.1888553018406-7.869342335684846e-15

Stress at elements
gauss-point sigma-11 sigma-22 sigma-12 sigma-33

Element no: 1
1 -7.028057140097936 0.8848825631177513 -0.11687298383729965 -1.8429523730940554
2 -3.1746036484995632 1.1249864906052225 -1.3463915376962121 -0.6148851473683022
3 -1.2161030019990122 2.9026457099131493 -1.004517397269694 0.5059628123742411
4 -3.1420923781383 0.9559079233176098 -1.1154639735442426 -0.6558553364462071
5 2.0388523903385445 -0.42842034989839983 -4.845535898915393 0.48312961213204336
6 2.517221823470721 -0.8571713072436549 -2.115789386742606 0.49801515486811987
7 0.8702303539581943 2.07643340335866 -0.2288230345393532 0.8839991271950562
8 1.2371144982499767 -1.3656645450834382 -0.44272538044152093 -0.03856501405003845
9 1.47677021882905 0.35126905420905613 0.3782049535042755 0.5484117819114318

Element no: 2
1 -1.156510117200924 1.21892365360414

5 1.5865339267389658 -1.0051335432666841 -0.22246888744807622 0.17442011504168448
6 1.8120185523340486 -0.3303165953802407 0.07946612046118252 0.4445105870861424
7 1.444079848882958 1.3136443613539521 0.06963543283260129 0.827317263071073
8 0.17772258842913013 -0.6187567624249941 0.5894762121612729 -0.1323102521987592
9 0.6897960381049815 0.6266865847426375 0.31997442613208676 0.3949447868542857

Element no: 6
1 -0.02156772987595801 0.7142901339231862 -0.21003612502521246 0.20781672121416844
2 0.25468002714833965 0.5032755365178063 -0.3360994517261817 0.22738666909984376
3 0.3252525484377154 0.7166032527994577 -0.5127421284203434 0.3125567403711519
4 0.5511058940805249 0.6334412269102019 -0.11031884896166598 0.355364136297218
5 1.0890159017680063 -0.10152045656378883 -0.08792566302504277 0.29624863356126524
6 1.1447864042830322 0.11308863090067361 0.055888186181043305 0.3773625105551117
7 1.0374093516970004 0.9360902730729477 0.024999907075691663 0.5920498874309844
8 -0.175714719135537

2 -3.1746036484995632 1.1249864906052225 -1.3463915376962121 -0.6148851473683022
3 -1.2161030019990122 2.9026457099131493 -1.004517397269694 0.5059628123742411
4 -3.1420923781383 0.9559079233176098 -1.1154639735442426 -0.6558553364462071
5 2.0388523903385445 -0.42842034989839983 -4.845535898915393 0.48312961213204336
6 2.517221823470721 -0.8571713072436549 -2.115789386742606 0.49801515486811987
7 0.8702303539581943 2.07643340335866 -0.2288230345393532 0.8839991271950562
8 1.2371144982499767 -1.3656645450834382 -0.44272538044152093 -0.03856501405003845
9 1.47677021882905 0.35126905420905613 0.3782049535042755 0.5484117819114318

Element no: 2
1 -1.156510117200924 1.2189236536041408 0.8917943710242293 0.018724060920965034
2 -1.1496236991232178 0.8618815998432949 -0.12263181568651366 -0.08632262978397687
3 -0.5477239195065782 1.7075806056681069 -0.3394500237579184 0.3479570058484586
4 0.0158881643369978 1.08108958614166 0.2306308450936528 0.3290933251435973
5 0.7703038452604376 -0.1360659

Element no: 6
1 -0.02156772987595801 0.7142901339231862 -0.21003612502521246 0.20781672121416844
2 0.25468002714833965 0.5032755365178063 -0.3360994517261817 0.22738666909984376
3 0.3252525484377154 0.7166032527994577 -0.5127421284203434 0.3125567403711519
4 0.5511058940805249 0.6334412269102019 -0.11031884896166598 0.355364136297218
5 1.0890159017680063 -0.10152045656378883 -0.08792566302504277 0.29624863356126524
6 1.1447864042830322 0.11308863090067361 0.055888186181043305 0.3773625105551117
7 1.0374093516970004 0.9360902730729477 0.024999907075691663 0.5920498874309844
8 -0.17571471913553743 -0.1723780602413193 0.4731817230204498 -0.10442783381305702
9 0.43979314139060766 0.4033671166215902 0.2449579558968227 0.2529480774036593

Element no: 7
1 -1.174656960443355 0.2796175143642271 -2.804862196318661 -0.26851183382373833
2 -0.7503460495597056 -2.5991641231080043 -0.7826427494986978 -1.004853051800313
3 1.2128050015246572 -0.6872572559762973 0.490791909002535 0.15766432366450797
4 0

6 2.517221823470721 -0.8571713072436549 -2.115789386742606 0.49801515486811987
7 0.8702303539581943 2.07643340335866 -0.2288230345393532 0.8839991271950562
8 1.2371144982499767 -1.3656645450834382 -0.44272538044152093 -0.03856501405003845
9 1.47677021882905 0.35126905420905613 0.3782049535042755 0.5484117819114318

Element no: 2
1 -1.156510117200924 1.2189236536041408 0.8917943710242293 0.018724060920965034
2 -1.1496236991232178 0.8618815998432949 -0.12263181568651366 -0.08632262978397687
3 -0.5477239195065782 1.7075806056681069 -0.3394500237579184 0.3479570058484586
4 0.0158881643369978 1.08108958614166 0.2306308450936528 0.3290933251435973
5 0.7703038452604376 -0.13606591734285012 -1.6014843802815337 0.19027137837527625
6 1.5692409833558902 0.0784093431405134 -1.0337379181372035 0.4942950979489211
7 1.3003551198274645 1.435721747402558 0.1425756977749648 0.8208230601690067
8 0.7770739529518643 -0.8613792153923906 -0.5293642261849725 -0.02529157873215788
9 0.9177287008832794 0.2728633

...

6 1.1447864042830322 0.11308863090067361 0.055888186181043305 0.3773625105551117
7 1.0374093516970004 0.9360902730729477 0.024999907075691663 0.5920498874309844
8 -0.17571471913553743 -0.1723780602413193 0.4731817230204498 -0.10442783381305702
9 0.43979314139060766 0.4033671166215902 0.2449579558968227 0.2529480774036593

Element no: 7
1 -1.174656960443355 0.2796175143642271 -2.804862196318661 -0.26851183382373833
2 -0.7503460495597056 -2.5991641231080043 -0.7826427494986978 -1.004853051800313
3 1.2128050015246572 -0.6872572559762973 0.490791909002535 0.15766432366450797
4 0.5831156412652427 0.8189366573632653 -1.1170202655861712 0.4206156895885524
5 1.7435548252658544 -2.303449572248736 2.212766040019484 -0.16796842409486448
6 2.158379656373093 0.6397688748197783 2.0124646296006476 0.8394445593578613
7 2.0580046651563557 2.230655038504766 -0.011689523641023969 1.2865979110983363
8 0.10142261246548634 -0.9539585784999145 1.947739693305702 -0.25576078981032846
9 2.2873742430700386 2.314

3 -0.5477239195065782 1.7075806056681069 -0.3394500237579184 0.3479570058484586
4 0.0158881643369978 1.08108958614166 0.2306308450936528 0.3290933251435973
5 0.7703038452604376 -0.13606591734285012 -1.6014843802815337 0.19027137837527625
6 1.5692409833558902 0.0784093431405134 -1.0337379181372035 0.4942950979489211
7 1.3003551198274645 1.435721747402558 0.1425756977749648 0.8208230601690067
8 0.7770739529518643 -0.8613792153923906 -0.5293642261849725 -0.02529157873215788
9 0.9177287008832794 0.27286334931860623 0.23118925679711833 0.35717761506056567

Element no: 3
1 -0.1945531714762873 0.7965880944866981 0.4260085377060818 0.1806104769031232
2 0.012193658964959941 0.8455861119945366 -0.0016298458337240954 0.2573339312878489
3 -0.029397754591452552 1.1661460944498818 -0.07037632948352854 0.34102450195752876
4 0.0667324923280298 0.6291314832482084 0.2259019513897167 0.20875919267287146
5 0.596711434413717 0.23495713369227464 -0.510839336685022 0.2495005704317975
6 1.0804176227780518 0.4

1 -1.174656960443355 0.2796175143642271 -2.804862196318661 -0.26851183382373833
2 -0.7503460495597056 -2.5991641231080043 -0.7826427494986978 -1.004853051800313
3 1.2128050015246572 -0.6872572559762973 0.490791909002535 0.15766432366450797
4 0.5831156412652427 0.8189366573632653 -1.1170202655861712 0.4206156895885524
5 1.7435548252658544 -2.303449572248736 2.212766040019484 -0.16796842409486448
6 2.158379656373093 0.6397688748197783 2.0124646296006476 0.8394445593578613
7 2.0580046651563557 2.230655038504766 -0.011689523641023969 1.2865979110983363
8 0.10142261246548634 -0.9539585784999145 1.947739693305702 -0.25576078981032846
9 2.2873742430700386 2.3147504920962665 -0.5069455411143067 1.3806374205498912

Element no: 8
1 0.26675119573129796 0.2667879862700218 -0.89880184471515 0.16006175460039593
2 0.37129529433548564 -0.3394352655009632 -0.44448258760942594 0.00955800865035673
3 0.7278518391966212 -0.013708534173154707 -0.4176735284028152 0.21424299150703993
4 0.8134339040973886 0.53

8 0.7770739529518643 -0.8613792153923906 -0.5293642261849725 -0.02529157873215788
9 0.9177287008832794 0.27286334931860623 0.23118925679711833 0.35717761506056567

Element no: 3
1 -0.1945531714762873 0.7965880944866981 0.4260085377060818 0.1806104769031232
2 0.012193658964959941 0.8455861119945366 -0.0016298458337240954 0.2573339312878489
3 -0.029397754591452552 1.1661460944498818 -0.07037632948352854 0.34102450195752876
4 0.0667324923280298 0.6291314832482084 0.2259019513897167 0.20875919267287146
5 0.596711434413717 0.23495713369227464 -0.510839336685022 0.2495005704317975
6 1.0804176227780518 0.41179833521263665 -0.4675670429434688 0.4476647873972065
7 0.3980964206898459 0.716562599537467 0.2496001148511276 0.3343977060681939
8 0.2908695420925223 -0.49297814038273324 0.05289105588432741 -0.06063257948706328
9 0.6961786332259561 0.1450641656262076 0.1299224689372079 0.2523728396556491

Element no: 4
1 -2.636284831893756 2.6539266517759215 -0.7600131381100008 0.005292545964649653
2 -1

8 0.17772258842913013 -0.6187567624249941 0.5894762121612729 -0.1323102521987592
9 0.6897960381049815 0.6266865847426375 0.31997442613208676 0.3949447868542857

Element no: 6
1 -0.02156772987595801 0.7142901339231862 -0.21003612502521246 0.20781672121416844
2 0.25468002714833965 0.5032755365178063 -0.3360994517261817 0.22738666909984376
3 0.3252525484377154 0.7166032527994577 -0.5127421284203434 0.3125567403711519
4 0.5511058940805249 0.6334412269102019 -0.11031884896166598 0.355364136297218
5 1.0890159017680063 -0.10152045656378883 -0.08792566302504277 0.29624863356126524
6 1.1447864042830322 0.11308863090067361 0.055888186181043305 0.3773625105551117
7 1.0374093516970004 0.9360902730729477 0.024999907075691663 0.5920498874309844
8 -0.17571471913553743 -0.1723780602413193 0.4731817230204498 -0.10442783381305702
9 0.43979314139060766 0.4033671166215902 0.2449579558968227 0.2529480774036593

Element no: 7
1 -1.174656960443355 0.2796175143642271 -2.804862196318661 -0.26851183382373833
2 